#Capstone Project (Pandas + Prediction)

Objective
Build a mini data-to-decision workflow: clean the dataset using Pandas, generate simple insights, then do two predictions one numeric (Regression) and one Yes/No (Classification).

1) Load and audit the data (Pandas)

In [88]:

import pandas as pd

df = pd.read_csv(r"C:\Users\sharanabasava\OneDrive\Desktop\3rd sem cls ppt\hospital_outpatient_DataSet 9-1-2026.csv")   # change filename


show outputs 

In [89]:
df.shape

(300, 13)

In [90]:
df.head()

,Visit_ID,Date,Department,Doctor_Name,Patient_Age,Gender,Symptoms,Diagnosis,Medicine_Cost,Consultation_Fee,Followup_Required,Visit_Duration,Total_Bill
0,V1170,01-01-2024,Cardiology,Dr. Ahmed,61.0,F,Allergy,Hypertension,1674.0,263.0,No,77,1937
1,V1221,01-01-2024,Cardiology,Dr. Ahmed,67.0,M,Fatigue,Migraine,764.0,359.0,Yes,62,1123
2,V1016,01-01-2024,Cardiology,Dr. Meena,48.0,F,Allergy,Fracture,1085.0,727.0,Yes,87,1812
3,V1205,03-01-2024,Orthopedics,Dr. Meena,34.0,F,NaN,Hypertension,1893.0,323.0,No,120,2216
4,V1212,03-01-2024,Orthopedics,Dr. Meena,29.0,F,Fever,Infection,1056.0,357.0,Yes,18,1413


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Visit_ID           300 non-null    object 
 1   Date               300 non-null    object 
 2   Department         300 non-null    object 
 3   Doctor_Name        300 non-null    object 
 4   Patient_Age        291 non-null    float64
 5   Gender             295 non-null    object 
 6   Symptoms           285 non-null    object 
 7   Diagnosis          285 non-null    object 
 8   Medicine_Cost      285 non-null    float64
 9   Consultation_Fee   293 non-null    float64
 10  Followup_Required  285 non-null    object 
 11  Visit_Duration     300 non-null    int64  
 12  Total_Bill         300 non-null    int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 30.6+ KB


In [92]:
df.isna().sum().sort_values(ascending=False)


Symptoms             15
Followup_Required    15
Medicine_Cost        15
Diagnosis            15
Patient_Age           9
Consultation_Fee      7
Gender                5
Date                  0
Visit_ID              0
Doctor_Name           0
Department            0
Visit_Duration        0
Total_Bill            0
dtype: int64

In [93]:
df.duplicated().sum()

np.int64(0)

The dataset was loaded successfully. Shape shows number of rows and columns. Info() displays data types and memory usage. Missing values and duplicate records were checked.

2.Cleaning (minimum 5 actions)

In [94]:
# 1️ Fix datatype (Date → datetime)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')



In [95]:
# 2️ Missing numeric values → Median
df['Patient_Age'] = df['Patient_Age'].fillna(df['Patient_Age'].median())
df['Medicine_Cost'] = df['Medicine_Cost'].fillna(df['Medicine_Cost'].median())


In [96]:

# 3️ Missing categorical values → Mode
df['Symptoms'] = df['Symptoms'].fillna(df['Symptoms'].mode()[0])

In [97]:
# 4️ Text cleaning
df['Department'] = df['Department'].str.strip().str.title()
df['Doctor_Name'] = df['Doctor_Name'].str.strip().str.title()
df['Gender'] = df['Gender'].str.strip().str.upper()


In [98]:
# 5️ Remove duplicates
df = df.drop_duplicates()

In [99]:
# 6️ Rename column (extra improvement)
df = df.rename(columns={'Patient_Age':'Age'})

In [100]:
df.isna().sum()


Visit_ID               0
Date                 195
Department             0
Doctor_Name            0
Age                    0
Gender                 5
Symptoms               0
Diagnosis             15
Medicine_Cost          0
Consultation_Fee       7
Followup_Required     15
Visit_Duration         0
Total_Bill             0
dtype: int64

Data cleaning included datatype conversion, handling missing values using median and mode, text formatting, duplicate removal, and column renaming.

3.Feature Engineering (Create 3 New Columns)


In [101]:
# 1️ Derived numeric column
df['Bill_Check'] = df['Medicine_Cost'] + df['Consultation_Fee']

In [102]:

# 2️ Conditional column
df['High_Bill_Flag'] = df['Bill_Check'] > 2000

In [103]:
# 3️ Bucket column
df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[0,25,40,60,100],
    labels=['Young','Adult','Middle','Senior']
)

In [104]:
df['Age_Group'] = df['Age_Group'].astype(str)


New features were created to support analysis and prediction.

4.Insights (Only 3 Required)

In [105]:
df.groupby('Department')['Bill_Check'].mean()


Department
Cardiology     1601.714286
Dermatology    1513.104167
Ent            1613.289474
General        1626.036364
Orthopedics    1660.921569
Name: Bill_Check, dtype: float64

In [106]:
df.sort_values('Bill_Check', ascending=False)[['Visit_ID','Bill_Check']].head()


,Visit_ID,Bill_Check
161,V1280,2791.0
288,V1192,2737.0
177,V1031,2641.0
49,V1145,2617.0
104,V1207,2589.0


In [107]:
df['Diagnosis'].value_counts()


Diagnosis
Fracture        61
Infection       59
Hypertension    57
Flu             55
Migraine        53
Name: count, dtype: int64

Cardiology has the highest average bill.
Top visits have bills above ₹3000.
 Most frequent diagnosis is Hypertension.

5.Regression (Predict Total Bill)
Predict numeric value
Target = Total_Bill

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [109]:
# Ensure Age_Group is categorical
df['Age_Group'] = df['Age_Group'].astype(str)

In [110]:
# Drop unwanted columns
X = df.drop([
    'Total_Bill',
    'Date',
    'Visit_ID',
    'Followup_Required'
], axis=1)

In [111]:
y = df['Total_Bill']

In [112]:
# Identify column types
cat_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(exclude='object').columns

5.Regression: Predict a number

In [113]:
# Pipelines for preprocessing
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

In [114]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [115]:
preprocess = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])


In [116]:
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [117]:
y = df['Total_Bill']


In [118]:
# Model pipeline
model = Pipeline([
    ('prep', preprocess),
    ('lr', LinearRegression())
])

In [119]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [120]:
# Train model
model.fit(X_train, y_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Age', 'Medicine_Cost', 'Consultation_Fee', 'Visit_Duration',
       'Bill_Check', 'High_Bill_Flag'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Department', 'Doctor_Name', 'Gender', 'Symptoms', 'Diagnosis',
       'Age_Group'],
      dtype='object'))])),
                ('lr', LinearRegression())])

In [121]:

# Prediction
pred = model.predict(X_test)


In [122]:

# MAE
mae = mean_absolute_error(y_test, pred)
mae

50.47906203216043

MAE output:

The Mean Absolute Error (MAE) is approximately 50, which means that on average the predicted hospital bill differs from the actual bill by about ₹50. Lower MAE indicates better prediction accuracy.

6.Classification: Predict Yes/No

In [123]:
# Remove rows where Followup_Required is missing
df = df.dropna(subset=['Followup_Required'])


In [124]:
y = df['Followup_Required']


In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


In [126]:
# Target
X = df.drop([
    'Followup_Required',
    'Date',
    'Visit_ID'
], axis=1)

In [127]:
y = df['Followup_Required']

In [128]:

# Ensure categorical column
X['Age_Group'] = X['Age_Group'].astype(str)

In [129]:
# Identify columns
cat_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(exclude='object').columns


In [130]:
 #Pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])


In [131]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [132]:
preprocess = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

In [133]:
# Logistic Regression model
clf_model = Pipeline([
    ('prep', preprocess),
    ('clf', LogisticRegression(max_iter=3000))
])


In [134]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [135]:
# Train
clf_model.fit(X_train, y_train)


C:\Users\sharanabasava\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Age', 'Medicine_Cost', 'Consultation_Fee', 'Visit_Duration',
       'Total_Bill', 'Bill_Check', 'High_Bill_Flag'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Department', 'Doctor_Name', 'Gender', 'Symptoms', 'Diagnosis',
       'Age_Group'],
      dtype='object'))])),
                ('clf', LogisticRegression(max_iter=3000))])

In [136]:
 #Predict
pred = clf_model.predict(X_test)


In [137]:

# Evaluation
confusion_matrix(y_test, pred)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          No       0.62      0.57      0.60        35
         Yes       0.40      0.45      0.43        22

    accuracy                           0.53        57
   macro avg       0.51      0.51      0.51        57
weighted avg       0.54      0.53      0.53        57



7.FINAL CONCLUSION

This capstone project applied Pandas-based data cleaning, transformation, and feature engineering on hospital outpatient data to build a reliable analytical workflow. Multiple cleaning techniques improved data quality and consistency. Insight analysis revealed important patterns across departments, billing behavior, and patient age groups. Linear regression achieved a Mean Absolute Error (MAE) of approximately 50, indicating accurate prediction of hospital bills. Logistic regression demonstrated moderate recall in identifying patients requiring follow-up, supporting decision-making in healthcare operations. Precision and recall metrics helped evaluate model reliability. A key limitation of this study is the relatively small dataset and limited feature availability, which may impact model generalization.


In [138]:
df.to_csv("cleaned_hospital_data.csv", index=False)
